In [1]:
import os
import xlsxwriter
import numpy as np
from pylab import plot, grid, xlim, ylim, xlabel, ylabel, linspace, figure, legend
import pandas as pd
from pandas import isnull

In [2]:
def CreateFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    return directory

def FindDel(S):
    """This function takes a the first string in a list,(S) that starts with ENG, it finds the nomancature 
    the file uses for the begining of each file, e.g ENG44007 or ENGINX00305258, it shall also work if more numbers
    are added or the name changes so long as ENG followed by anything other than "." or "_" or " " is still 
    the start of each file"""
    for i in S:
        if i[0:3] == 'ENG':
            n = 1
            for j in i:
                if j == '.':
                    break
                elif j == ' ':
                    break
                elif j == '_':
                    break
                else: 
                    n += 1 
            return n
        else:
            continue


def InitialRun(S):
    """This takes S a list of filenames and returns the first run index"""
    last_int_index = FindDel(S) -1
    first_int_index = 0 
    for i in S:                                                          
        if i[0:3] == 'ENG':
            for j in range(last_int_index):
                try:
                    int(i[j])
                    run_number = i[first_int_index:last_int_index]
                    break
                except:
                    first_int_index += 1
        else:
            continue
        break
    return((run_number))

def CycleRuns(S):
    """This takes a list of filenames and returns a list of all the runs in the file"""
    Start_run = int(InitialRun(S))
    Last_run = int(InitialRun(reversed(S)))
    Runs = range(Start_run,Last_run)
    return (list(Runs))

def RunIndex(s,n): #ENGINX00305258
    """s is a string in the for m of a filename from the enginx data and returns it's run number, n is the FindDel"""
    last_int_index = n-1
    first_int_index = 0
    for j in range(last_int_index):
        try:
            int(s[j])
            return s[first_int_index:last_int_index]
        except:
            first_int_index += 1

### Making spreadsheet of file names 

In [ ]:
directory = "D:\\"
array_names = []
file_years = []
long_array_names = []
for foldername in os.listdir(directory):
    if foldername[0:5] == 'cycle':
        n = FindDel(os.listdir("D:\\"+foldername))
        print(n)
        for filename in os.listdir("D:\\"+foldername):
            if filename[n:] in array_names: #stop repeated file names from entering 
                pass
            elif len(filename) < n+1: # Stop useless file names from entering (e.g copy)
                pass
            elif filename[n] in ("S", "s", "N", "n"):# Stop temporary files from entering (e.g .s01 or .n001)
                try:
                    int(filename[n+1]) # all temp files are 's' or 'n' followed by a int
                    temp_files.append(filename)
                except ValueError: 
                    array_names.append(filename[n:])
                    long_array_names.append(filename)
                    file_years.append(foldername)
            else: # most unique file names go here 
                array_names.append(filename[n:])
                long_array_names.append(filename)
                file_years.append(foldername)
                
#print(file_years)
#print(array_names)
#print(long_array_names)

workbook   = xlsxwriter.Workbook('Filenames.xlsx')

worksheet1 = workbook.add_worksheet()
worksheet2 = workbook.add_worksheet()

worksheet1.write_column('A1', array_names)
worksheet2.write_column('A1', long_array_names)
worksheet1.write_column('B1', file_years)
worksheet2.write_column('B1', file_years)
workbook.close()

### Extracting times

In [ ]:
directory = "D:\\"
time = np.array([])
Experiment_index = np.array([])
for foldername in os.listdir(directory):
    if foldername[0:5] == 'cycle':
        n = FindDel(os.listdir("D:\\"+foldername))
        print(n)
        for filename in os.listdir("D:\\"+foldername):
            if filename[-4:] == ".txt":
                file_path = directory+foldername+"\\"+filename
                df = pd.read_csv(file_path, sep = "\t", names = ["Date\time", "other"])
                li = df["Date\time"].values
                if os.stat(file_path).st_size == 0: # remove empty files
                    pass
                elif len(str(df.iloc[0].values[0])) < 18: # remove the files without time in the first colmn
                    print(filename)
                    pass
                elif li[0][10] =="T":
                    for i in range(len(li)): # fix the T issue in some times 
                        untdate = li[0].replace('T',' ')
                        li = np.append(li,[untdate])
                        li = np.delete(li,[0], axis = None)
                    time = np.concatenate((a,li),axis=None)
                    Experiment_index = np.append(Experiment_index,[filename[:n-1]])
                else: # Normal time columns
                    time = np.concatenate((a,li),axis=None)
                    Experiment_index = np.append(Experiment_index,[filename[:n-1]])
                    #print(a)
print(len(time))
print(time)
print(Experiment_index)
print(len(Experiment_index))

### Making a dictionary from spreadsheet of filenames

In [3]:
path = "C:\\Users\\zahaab\\ENGIN_X_DATA_CLEANING\\File_names_2013-19.xlsx"
def Translator(path):
    xls = pd.ExcelFile(path)
    df = xls.parse(sheet_name = "Sheet1")
    df = df[["Filename", "Column_name"]]
    df = df.fillna(0)
    true_name_translater = dict(zip(df["Filename"],df["Column_name"]))
    s = set(true_name_translater.values())
    col_names = list(s)
    col_names.remove(0)
    col_names.append("measured_temperature_U")
    col_names.append("unregistered_value")
    return true_name_translater, col_names

true_name_translater, col_names = Translator(path)

### Extracting data from raw files 

In [4]:
def AddRawData(run_number):
    file_path = output+"\\"+str(run_number)+".txt"
    output_file = open(file_path, "r")
    contents = output_file.read()
    split_contents = contents.split("kamehameha") # Had to pick somthing that wouldn't be in data
    del split_contents[-1]
    clean_split_contents = []
    for i,j in zip(split_contents, data):
        a = i.replace("\n","").replace("[","").replace("]","")
        a = a.split(" ")
        a = list(filter(None, a))
        if "data" in j:
            clean_split_contents.append(j[0:16]+str(run_number))
            Neutron_file = open(Neutron_folder+"\\"+j[0:16]+str(run_number)+".txt", "w+")
            Neutron_file.write(str(a))
            Neutron_file.close()
        elif "time" in j:
            untdate = i.replace('T',' ')
            clean_split_contents.append(untdate)
        else:
            clean_split_contents.append(a)
    raw_data = dict(zip(data, list(clean_split_contents)))
    output_file.close()
    #os.remove(file_path) Turn on when acctualy running
    return raw_data

In [7]:
def NeutronDataTranslator(Neutron_key):
    Neutron_file = open(Neutron_folder+"\\"+Neutron_key+".txt", "r")
    contents = Neutron_file.read()
    contents = contents.replace("'","").replace("[","").replace("]","")
    contents = contents.split(", ")
    contents = list(map(float, contents))
    Neutron_file.close
    return contents

In [6]:
output= r"C:\Users\zahaab\temp_raw_data"
Neutron_folder = r"C:\Users\zahaab\Neutron_data"
data = ["North_neutron_X_data","North_neutron_Y_data","South_neutron_X_data",
            "South_neutron_Y_data","neutron_start_time","neutron_end_time","Run_Title"] #neutron data col names
raw_data = AddRawData(44296)
print(raw_data)
North_neutron_X_44296 = NeutronDataTranslator("North_neutron_X_44296")
print(North_neutron_X_44296)
print(type(North_neutron_X_44296))

{'North_neutron_X_data': 'North_neutron_X_44296', 'North_neutron_Y_data': 'North_neutron_Y_44296', 'South_neutron_X_data': 'South_neutron_X_44296', 'South_neutron_Y_data': 'South_neutron_Y_44296', 'neutron_start_time': '2004-10-28 18:21:36 \n', 'neutron_end_time': '2004-10-28 18:22:23 \n', 'Run_Title': ['yscan;y=-246.889999999999']}
[0.29232276, 0.31129013, 0.33025749, 0.34922486, 0.36819222, 0.38715959, 0.40612695, 0.42509432, 0.44406168, 0.46302905, 0.48199641, 0.50096378, 0.51993114, 0.53889851, 0.55786587, 0.57683324, 0.59580061, 0.61476797, 0.63373534, 0.6527027, 0.67167007, 0.69063743, 0.7096048, 0.72857216, 0.74753953, 0.76650689, 0.78547426, 0.80444162, 0.82340899, 0.84237635, 0.86134372, 0.88031108, 0.89927845, 0.91824581, 0.93721318, 0.95618055, 0.97514791, 0.99411528, 1.01308264, 1.03205001, 1.05101737, 1.06998474, 1.0889521, 1.10791947, 1.12688683, 1.1458542, 1.16482156, 1.18378893, 1.20275629, 1.22172366, 1.24069102, 1.25965839, 1.27862575, 1.29759312, 1.31656049, 1.335527

### Making function(s) for extracting data from files 

In [29]:
def NewRun(run_number, run_data, time_list, raw_data, file_location):
    """This activates if a new run has started, it makes a new list of dictionaries and makes a CSV out of current data"""
    df1 = pd.DataFrame(columns = ["Date/Time"])
    for i in time_list:
        row = np.array([i])
        column_names = np.array(["Date/Time"])
        for j in run_data:
            try:
                row = np.append(row, [j[1][i]])
                column_names = np.append(column_names, [j[0]])
            except:
                pass
        df2 = pd.DataFrame([row], columns = column_names)
        df1 = df1.append(df2, sort=False)
    df1.set_index("Date/Time", drop=True)
    for i in raw_data:
        df1[str(i)] = str(raw_data[i])
    print(run_number)
    print(file_location)
    df1.to_csv(file_location+"\\"+run_number)
    
    raw_data = {}
    run_data = []
    time_list = np.array([])
    for i,j in enumerate(col_names):
        run_data.append((j,{}))
    
    return run_data,time_list,raw_data

def Find_dict(run_data, trans_filename):
    for i,j in enumerate(run_data):
        if j[0] == trans_filename:
            return i

def AddData(file_path, filename, n, run_data, time_list):
    """This takes data from a txt file and puts it into a list of times and the dictionaries"""
    df = pd.read_csv(file_path, sep = "\t", names = ["Date/Time", str(true_name_translater[filename[n:]])])
    file_times = df["Date/Time"].values
    file_data = df[str(true_name_translater[filename[n:]])].values 
    dict_index = Find_dict(run_data, true_name_translater[filename[n:]])
    if os.stat(file_path).st_size == 0: # remove empty files
        pass
    elif len(str(df.iloc[0].values[0])) < 18: # remove the files without time in the first colmn
        pass
    elif file_times[0][10] =="T":
        for i in range(len(li)): # fix the T issue in some times 
            untdate = file_times[0].replace('T',' ')
            file_times = np.append(file_times,[untdate])
            file_times = np.delete(file_times,0, axis = None)
        time_list = np.unique(np.append(time_list, file_times))
        for i, j in zip(file_times,file_data):
            if "temperature" in true_name_translater[filename[n:]]:#Alot of tempreture files, may be some overlap in time
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-2][1][i] = j
                except:
                    run_data[dict_index][1][i] = j
            else:#rest of the files 
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-1][1][i] = [run_data[dict_index][0],j]
                except:
                    run_data[dict_index][1][i] = j
    else: # Normal time columns
        time_list = np.unique(np.append(time_list, file_times))
        for i, j in zip(file_times,file_data):
            if "temperature" in true_name_translater[filename[n:]]:
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-2][1][i] = j
                except:
                    run_data[dict_index][1][i] = j
            else:
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-1][1][i] = [run_data[dict_index][0],j]
                except:
                    run_data[dict_index][1][i] = j
    return run_data, time_list


In [30]:
%%timeit
%prun
direc = "E:\\"
output= r"E:\temp_raw_data"
Neutron_folder = r"E:\Neutron_data"
ENGINX_DATA = r"E:\clean_data"
data = ["North_neutron_X_data","North_neutron_Y_data","South_neutron_X_data",
            "South_neutron_Y_data","neutron_start_time","neutron_end_time","Run_Title"] #neutron data col names


for foldername in os.listdir(direc):
    if foldername[0:5] == 'cycle':
        if foldername != "cycle_04_1":#just for test
            break
        folder_path = direc+"\\"+foldername
        clean_data = CreateFolder(ENGINX_DATA+"\\"+foldername)
        print(clean_data)
        n = FindDel(os.listdir(folder_path))
        filenames = os.listdir(folder_path)
        raw_data = {}
        run_data = []
        for i,j in enumerate(col_names):
            run_data.append((j,{}))
        time_list = np.array([])
        finale_file = 0
        for k, filename in enumerate(filenames):
            print(filename)
            file_path = folder_path+'\\'+filename
            if "Copy" in filename:
                pass 
            elif RunIndex(filenames[k],n) != RunIndex(filenames[k-1],n):
                if len(time_list) == 0:
                    pass
                else:
                    run_data,time_list,raw_data = NewRun(RunIndex(filenames[k-1],n), run_data, time_list, raw_data, clean_data)
                    if filename[-3:].lower() == "raw":
                        raw_data = AddRawData(RunIndex(filenames[k],n))
                    elif filename[-3:] == "txt":
                        run_data,time_list = AddData(file_path, filename, n, run_data, time_list)
                    else:
                        pass
            else:
                if filename[-3:].lower() == "raw":
                    raw_data = AddRawData(RunIndex(filenames[k],n))
                elif filename[-3:] == "txt":
                    run_data,time_list = AddData(file_path, filename, n, run_data, time_list)
                else:
                    pass
            finale_file = filenames[k]
        NewRun(RunIndex(filenames[k],n), run_data, time_list, raw_data, clean_data)

 E:\clean_data\cycle_04_1
Copy of ENG45688.RAW
Copy of ENG45995_Strain.txt
copy.log
ENG44007.RAW
ENG44007.S01
ENG44007_Current.txt
ENG44007_dash_controls.txt
ENG44007_MonCounts.txt
ENG44007_Status.txt
ENG44007_Theta.txt
ENG44007_X.txt
ENG44007_Y.txt
ENG44007_Z.txt


KeyboardInterrupt: 

### Finding how to deal with Stress rig data (work in progress)


In [9]:
direct = "E:\\stuff\\OtherData\\cycle_13_1\\Stress Rig"

for filename in os.listdir(direct):
    #print(filename)
    file_path = direct+"\\"+filename
    #print(filename)
    rig_data = pd.read_csv(file_path, sep = "|", skipinitialspace = True, skiprows = [0,1])
    output_file = open(file_path, "r")
    first_line = output_file.readline()
    constants = {"Cross_Sectional_Area" : first_line[:33].split("=")[-1],
                 "Gauge_Length_for_Strain1" : first_line[33:].split("=")[-1]}
    for i in constants:
        rig_data[str(i)] = str(constants[i]).replace("\n","")
    #print(rig_data)
        
rig_data

,Date/Time,Position,Averaged Position (mm),Load,Averaged Load (MPa),Strain 1,Averaged Strain 1 (%),Cross_Sectional_Area,Gauge_Length_for_Strain1
0,2013/07/15 14:38:36,11.674732,11.674963,-0.318130,-0.321567,0.007048,0.007356,50.643180,12.500000
1,2013/07/15 14:38:37,11.675316,11.674941,-0.260695,-0.312952,0.006648,0.007188,50.643180,12.500000
2,2013/07/15 14:38:38,11.672592,11.674448,-0.435002,-0.356782,0.007064,0.007048,50.643180,12.500000
3,2013/07/15 14:38:39,11.674756,11.674440,-0.293532,-0.330865,0.007173,0.007054,50.643180,12.500000
4,2013/07/15 14:38:40,11.674589,11.674397,-0.478078,-0.357088,0.007288,0.007044,50.643180,12.500000
5,2013/07/15 14:38:41,11.673325,11.674116,-0.501147,-0.393691,0.007343,0.007103,50.643180,12.500000
6,2013/07/15 14:38:42,11.674273,11.673907,-0.469957,-0.435543,0.007599,0.007293,50.643180,12.500000
7,2013/07/15 14:38:43,11.673874,11.674164,-0.226917,-0.393926,0.006889,0.007258,50.643180,12.500000
8,2013/07/15 14:38:44,11.674190,11.674050,-0.240334,-0.383287,0.007788,0.007381,50.643180,12.500000
9,2013/07/15 14:38:45,11.672050,11.673542,-0.374624,-0.362596,0.007718,0.007467,50.643180,12.500000


In [ ]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                    index=[0, 1, 2, 3]) 

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[4, 5, 6, 7])

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                    index=[8, 9, 10, 11])

df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                    index=[2, 3, 6, 7])

result = result = pd.concat([df1, df4], axis=1, sort=False)

result

In [27]:
def CombineRigCleanData(cycle):
    direct = "E:\\stuff\\OtherData\\"+cycle+"\\Stress Rig"
    
    df1 = pd.DataFrame()
    for filename in os.listdir(ENGINX_DATA+"\\"+cycle):
        file_path = ENGINX_DATA+"\\"+cycle+"\\"+filename
        df2 = pd.read_csv(file_path,index_col=0)
        df2["Run_Number"] = filename
        df1 = df1.append(df2, ignore_index=False, sort=False)
    time_list = df1["Date/Time"].values
    df1.set_index("Date/Time") 

    df3 = pd.DataFrame()
    for filename in os.listdir(direct):
        file_path = direct+"\\"+filename
        df4 = pd.read_csv(file_path, sep = "|", skipinitialspace = True, skiprows = [0,1])
        df3 = df3.append(df4, ignore_index=False, sort=False)
        input_file = open(file_path+".txt", "r")
        first_line = output_file.readline()
        constants = {"Cross_Sectional_Area" : first_line[:33].split("=")[-1].replace(" ", ""),
                     "Gauge_Length_for_Strain1" : first_line[33:].split("=")[-1].replace(" ", "")}
        input_file.close()
        for i in constants:
            df3[str(i)] = str(constants[i]).replace("\n","")

    df3 = df3.rename(index = str, columns = dict(zip(
        df3.columns.values.tolist(), ['rig_{0}'.format(i).replace(" ", "") for i in df3.columns.values.tolist()])))
    df3 = df3.rename(index = str, columns = {'rig_Date/Time' : 'Date/Time'})

    #df3.set_index("Date/Time")
    #df3 = df3.loc[~df3.index.duplicated(keep='first')] #drop duplicates 
    df3 = df3.loc[~df3["Date/Time"].duplicated(keep='first')]
    df5 = df3.loc[df3["Date/Time"].isin(time_list)]
    result = pd.concat([df5, df1], axis=1, sort=False, join='outer')

    return(result) 

def CleanInstron01(cycle, filenames, clean_file_time):
    direct = r"E:\stuff\OtherData\INSTRON_01"
    
    df1 = pd.DataFrame()
    for filename in os.listdir(ENGINX_DATA+"\\"+cycle):
        file_path = ENGINX_DATA+"\\"+cycle+"\\"+filename
        df2 = pd.read_csv(file_path,index_col=0)
        df2["Run_Number"] = filename
        df1 = df1.append(df2, ignore_index=False, sort=False)
    time_list = df1["Date/Time"].values
    df1.set_index("Date/Time") 
    
    #find the files that needed for the spesific cycle
    start_file_index, end_file_index = 0, 0
    for i in clean_file_time:
        if i in time_list:
            start_file_index = clean_file_time.index(i)-1
            break 
    for i in clean_file_time.reverse():
        if i in time_list:
            end_file_index = (clean_file_time.index(i)-1)*-1
            break 
    
    df3 = pd.DataFrame()
    for filename in os.listdir(direct)[start_file_index, end_file_index+1]:
        file_path = direct+"\\"+filename
        df4 = pd.read_csv(file_path, sep = "\t", skipinitialspace = True, skiprows = [0,1,2,3])
        df3 = df3.append(df4, ignore_index=False, sort=False)
        input_file = open(file_path+".txt", "r")
        first_line, second_line, third_line = output_file.readline(), output_file.readline(1), output_file.readline(2)
        constants = {"Cross_Sectional_Area" : first_line.split("=")[-1].replace(" ", ""),
                     "Gauge_Length_for_Strain1" : second_line.split("=")[-1].replace(" ", ""),
                     "RB Number" : third_line.split("=")[-1].replace(" ", "")}
        input_file.close()
        for i in constants:
            df3[str(i)] = str(constants[i]).replace("\n","")
        
    df3 = df3.rename(index = str, columns = dict(zip(
        df3.columns.values.tolist(), ['rig_{0}'.format(i).replace(" ", "") for i in df3.columns.values.tolist()])))
    df3 = df3.rename(index = str, columns = {'rig_Date/Time' : 'Date/Time'})
    df3 = df3.loc[~df3["Date/Time"].duplicated(keep='first')]
    
    unttime = []
    for i in df3["Date/Time"].values:
        untdate = "{}-{}-{} {}:{}:{}".format(
            date[0:4], date[5:7], date[8:10], date[11:13], date[14:16], date[17:19])
        unttime.append(untdate)
    df3 = df3.drop(columns=["Date/Time"])
    df3["Date/Time"] = unttime
    
    df5 = df3.loc[df3["Date/Time"].isin(time_list)]
    result = pd.concat([df5, df1], axis=1, sort=False, join='outer')
    return(result) 

In [26]:
ENGINX_DATA = r"E:\clean_data"

instron_start = len(os.listdir(ENGINX_DATA))

filenames = os.listdir(r"E:\stuff\OtherData\INSTRON_01")
clean_file_time = []
for filename in filenames:
    if "continuous" in filename:
        filenames.remove(filename)
    else: 
        date = filename[11:]
        untdate = "{}-{}-{} {}:{}:{}".format(
        date[0:4], date[5:7], date[8:10], date[11:13], date[14:16], date[17:19])
        clean_file_time.append(untdate)
            
for i,j in  enumerate(os.listdir(ENGINX_DATA)):
    if j == "cycle_17_3":
        CombineRigCleanData(j)
        instron_start = i
    elif i > instron_start:
        CleanInstron01(j, filenames, clean_file_time)
    else:
        CombineRigCleanData(j)

KeyboardInterrupt: 

In [24]:
li = ["a","b","c"]
for i,j in enumerate(li):
    print(i,j)

0 a
1 b
2 c
